## 版本
    
    不同的版本可能需要不同的处理方式，这时需要请求中包含版本信息，以便视图函数对请求进行正确处理。版本信息可以包含在多个地方，如请求url、get参数形式、请求头的version、请求头的Host，甚至可以通过路由分发的namespace参数值进行确定。对于这些方式，drf均提供了相应的处理类
    
    版本信息一般会包含在请求url中。含两种方式：
    1.以get参数形式传入，如：/api/...?version=v1
    2.包含在url路径中，如：/api/v1/....
    
    更多的是采用包含在url路径的方式。
    
### 1.流程 - self.determine_version

    发生节点：对request进一步封装后，进行用户、权限、节流认证前。
    
    流程：
    1.获取视图类中设定的版本解析器，如果视图类中未配置，则会从settings中加载
    2.实例化解析器，调用实例的determine_version方法获取版本号，并赋值给request.version, 将解析器对象赋值给request.versioning_scheme
    
#### self.determine_version(request, *args, **kwargs)源码

In [2]:
def determine_version(self, request, *args, **kwargs):
    """
    If versioning is being used, then determine any API version for the
    incoming request. Returns a two-tuple of (version, versioning_scheme)
    """
    if self.versioning_class is None:
        return (None, None)
    scheme = self.versioning_class()
    return (scheme.determine_version(request, *args, **kwargs), scheme)

# request.version, request.versioning_scheme = self.determine_version(...)   

# versioning_class = api_settings.DEFAULT_VERSIONING_CLASS

### 2.demo配置实例 - 以URLPathVersioning为例

#### URLPathVersioning.determine_version()

In [3]:
# 从url解析器传入的kwargs中获取self.version_param的值,倘若没有，则设置默认值self.default_version
# 检查获取到的版本号是否合法，即是否位于self.allowed_versions列表中

def determine_version(self, request, *args, **kwargs):
    version = kwargs.get(self.version_param, self.default_version)
    if version is None:
        version = self.default_version

    if not self.is_allowed_version(version):
        raise exceptions.NotFound(self.invalid_version_message)
    return version


# BaseVersioning.is_allowed_version()
def is_allowed_version(self, version):
    if not self.allowed_versions:
        return True
    return ((version is not None and version == self.default_version) or
            (version in self.allowed_versions))

#### url中的命名组

    无论是局部配置还是全局配置，通过URLPathVersioning获取版本号，则需要设计正则表达式进行匹配。其命名组表达式为：(?P<version>[v1|v2]+)
     
    举例如下：
    ￥代表美元符号，直接使用美元符号会导致jupyter格式错乱
    urlpatterns = [
        url(r'^(?P<version>[v1|v2]+)/users/￥', users_list, name='users-list'),
        url(r'^(?P<version>[v1|v2]+)/users/(?P<pk>[0-9]+)/￥', users_detail, name='users-detail')
    ]

#### 2.1局部配置


In [ ]:
from rest_framework.versioning import URLPathVersioning

class MyView(APIView):
    
    versioning_class = UrlPathVersioning   # 版本解析类
    version_param = 'version'              # url中版本参数的名称，用于获取url中匹配到的版本号
    default_version = 'v1'                 # 默认版本
    allowed_version = ['v1', 'v2']         # 合法的版本
    
    def get(self, request):
        pass
    

#### 2.2 全局配置


In [5]:
# settings.py

REST_FRAMEWORK = {
    # 版本配置
    'DEFAULT_VERSION': 'v1',
    'ALLOWED_VERSIONS': ['v1', 'v2'],
    'VERSION_PARAM': 'version',
    'DEFAULT_VERSIONING_CLASS': 'rest_framework.versioning.URLPathVersioning',
    
    # 认证配置
    'DEFAULT_AUTHENTICATION_CLASSES': ['api.utils.auth.UserAuthentication']，
    
    # 权限配置
    'DEFAULT_PERMISSION_CLASSES': ['api.utils.auth.UserPermission'],
    
    # 节流配置
    'DEFAULT_THROTTLE_RATES': {
        'user_throttle': '1/s'
    }
}